In [1]:
# Lightweight Blockchain Crawler & Listener
# pip install web3 pandas matplotlib seaborn ipywidgets
# Project by: [Daniel Folino, Partner Name]

import os
import json
import time
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from web3 import Web3
import sqlite3
from IPython.display import display, clear_output
import ipywidgets as widgets
import threading

# ==================== CONFIGURATION ====================
INFURA_API_KEY = "bbd4542ad25c457fad9cfb134b599242"  # https://infura.io/
INFURA_URL = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
DB_FILE = "blockchain_data.db"

# ==================== BLOCKCHAIN CONNECTION ====================
def connect_to_ethereum():
    """Connect to Ethereum network using HTTPProvider."""
    try:
        web3 = Web3(Web3.HTTPProvider(INFURA_URL))
        if web3.is_connected():
            print("Successfully connected to Ethereum Sepolia Testnet")
            print(f"Current block number: {web3.eth.block_number}")
            return web3
        else:
            print("Failed to connect to Ethereum network")
            return None
    except Exception as e:
        print(f"Error connecting to Ethereum network: {e}")
        return None

# ==================== DATABASE MANAGEMENT ====================
def setup_database():
    """Create SQLite database with tables for blocks and transactions."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # Create blocks table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS blocks (
        block_number INTEGER PRIMARY KEY,
        timestamp INTEGER,
        miner TEXT,
        difficulty INTEGER,
        total_transactions INTEGER,
        gas_used INTEGER,
        gas_limit INTEGER,
        date_crawled TIMESTAMP
    )
    ''')
    
    # Create transactions table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        tx_hash TEXT PRIMARY KEY,
        block_number INTEGER,
        from_address TEXT,
        to_address TEXT,
        value REAL,
        gas INTEGER,
        gas_price INTEGER,
        timestamp INTEGER,
        date_crawled TIMESTAMP,
        FOREIGN KEY (block_number) REFERENCES blocks (block_number)
    )
    ''')
    
    conn.commit()
    conn.close()
    print("Database setup complete")
    return True

# ==================== BLOCKCHAIN CRAWLER FUNCTIONS ====================
def fetch_block_data(web3, block_number):
    """Fetch data for a specific block."""
    try:
        # Get block data (with full transaction objects)
        block = web3.eth.get_block(block_number, full_transactions=True)
        
        # Extract relevant block information with explicit type conversions
        block_data = {
            'block_number': int(block.number),
            'timestamp': int(block.timestamp),
            'miner': str(block.miner),
            'difficulty': int(block.difficulty),
            'total_transactions': len(block.transactions),
            'gas_used': int(block.gasUsed),
            'gas_limit': int(block.gasLimit),
            'date_crawled': datetime.now().isoformat()  # store as string
        }
        
        # Extract transaction information with type conversions
        transactions = []
        for tx in block.transactions:
            tx_data = {
                'tx_hash': tx.hash.hex(),
                'block_number': int(block.number),
                'from_address': tx['from'],
                'to_address': tx['to'] if tx['to'] else 'Contract Creation',
                'value': float(web3.from_wei(tx.value, 'ether')),  # convert to float
                'gas': int(tx.gas),
                'gas_price': int(tx.gasPrice),
                'timestamp': int(block.timestamp),
                'date_crawled': datetime.now().isoformat()  # store as string
            }
            transactions.append(tx_data)
            
        return block_data, transactions
    
    except Exception as e:
        print(f"Error fetching block {block_number}: {e}")
        return None, None

def store_block_data(block_data, transactions):
    """Store block and transaction data in SQLite database."""
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        
        # Insert block data
        cursor.execute('''
        INSERT OR REPLACE INTO blocks
        (block_number, timestamp, miner, difficulty, total_transactions, gas_used, gas_limit, date_crawled)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            block_data['block_number'], 
            block_data['timestamp'], 
            block_data['miner'], 
            block_data['difficulty'], 
            block_data['total_transactions'], 
            block_data['gas_used'], 
            block_data['gas_limit'], 
            block_data['date_crawled']
        ))
        
        # Insert transaction data
        for tx in transactions:
            cursor.execute('''
            INSERT OR REPLACE INTO transactions
            (tx_hash, block_number, from_address, to_address, value, gas, gas_price, timestamp, date_crawled)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                tx['tx_hash'],
                tx['block_number'],
                tx['from_address'],
                tx['to_address'],
                tx['value'],
                tx['gas'],
                tx['gas_price'],
                tx['timestamp'],
                tx['date_crawled']
            ))
        
        conn.commit()
        conn.close()
        return True
    
    except Exception as e:
        print(f"Error storing data: {e}")
        return False

def crawl_latest_blocks(web3, num_blocks=10):
    """Crawl the latest blocks and store their data."""
    latest_block = web3.eth.block_number
    start_block = max(0, latest_block - num_blocks + 1)
    
    print(f"Crawling blocks {start_block} to {latest_block}...")
    
    for block_num in range(start_block, latest_block + 1):
        print(f"Processing block {block_num}...")
        block_data, transactions = fetch_block_data(web3, block_num)
        if block_data and transactions:
            success = store_block_data(block_data, transactions)
            if success:
                print(f"Block {block_num} with {len(transactions)} transactions stored successfully")
            else:
                print(f"Failed to store data for block {block_num}")
        time.sleep(0.5)  # Be nice to the network/API
    
    print("Crawling complete!")
    
# ==================== DATA ANALYSIS AND VISUALIZATION ====================
def load_blockchain_data():
    """Load blockchain data from SQLite database into pandas DataFrames."""
    conn = sqlite3.connect(DB_FILE)
    blocks_df = pd.read_sql_query("SELECT * FROM blocks", conn)
    if not blocks_df.empty:
        blocks_df['datetime'] = pd.to_datetime(blocks_df['timestamp'], unit='s')
    tx_df = pd.read_sql_query("SELECT * FROM transactions", conn)
    if not tx_df.empty:
        tx_df['datetime'] = pd.to_datetime(tx_df['timestamp'], unit='s')
    conn.close()
    return blocks_df, tx_df

def visualize_block_data(blocks_df):
    """Visualize block data."""
    if blocks_df.empty:
        print("No block data available for visualization")
        return
    
    plt.style.use('ggplot')
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Transactions per Block
    axs[0, 0].bar(blocks_df['block_number'], blocks_df['total_transactions'], color='blue', alpha=0.7)
    axs[0, 0].set_title('Transactions per Block')
    axs[0, 0].set_xlabel('Block Number')
    axs[0, 0].set_ylabel('Number of Transactions')
    
    # 2. Gas Used vs. Gas Limit
    axs[0, 1].plot(blocks_df['block_number'], blocks_df['gas_used'], 'r-', label='Gas Used')
    axs[0, 1].plot(blocks_df['block_number'], blocks_df['gas_limit'], 'b--', label='Gas Limit')
    axs[0, 1].set_title('Gas Used vs. Gas Limit')
    axs[0, 1].set_xlabel('Block Number')
    axs[0, 1].set_ylabel('Gas')
    axs[0, 1].legend()
    
    # 3. Gas Utilization Percentage
    blocks_df['gas_utilization'] = (blocks_df['gas_used'] / blocks_df['gas_limit']) * 100
    axs[1, 0].bar(blocks_df['block_number'], blocks_df['gas_utilization'], color='green', alpha=0.7)
    axs[1, 0].set_title('Gas Utilization Percentage')
    axs[1, 0].set_xlabel('Block Number')
    axs[1, 0].set_ylabel('Utilization (%)')
    
    # 4. Top 10 Miners (by frequency)
    if len(blocks_df) > 1:
        miner_counts = blocks_df['miner'].value_counts().head(10)
        miner_counts.plot(kind='pie', ax=axs[1, 1], autopct='%1.1f%%', startangle=90)
        axs[1, 1].set_title('Top 10 Miners')
    else:
        axs[1, 1].text(0.5, 0.5, 'Not enough data', horizontalalignment='center', verticalalignment='center')
    
    plt.tight_layout()
    plt.show()

def visualize_transaction_data(tx_df):
    """Visualize transaction data."""
    if tx_df.empty:
        print("No transaction data available for visualization")
        return
    
    plt.style.use('ggplot')
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Transaction Value Distribution (log scale)
    tx_values = tx_df['value'].replace(0, 0.000001)  # Avoid log(0)
    axs[0, 0].hist(tx_values, bins=30, color='purple', alpha=0.7, log=True)
    axs[0, 0].set_title('Transaction Value Distribution (log scale)')
    axs[0, 0].set_xlabel('Value (ETH)')
    axs[0, 0].set_ylabel('Frequency (log)')
    
    # 2. Gas Price Over Time
    axs[0, 1].scatter(tx_df['block_number'], tx_df['gas_price'] / 1e9, alpha=0.5, color='orange')
    axs[0, 1].set_title('Gas Price Over Time')
    axs[0, 1].set_xlabel('Block Number')
    axs[0, 1].set_ylabel('Gas Price (Gwei)')
    
    # 3. Gas Used by Transactions
    axs[1, 0].hist(tx_df['gas'], bins=30, color='green', alpha=0.7)
    axs[1, 0].set_title('Gas Used by Transactions')
    axs[1, 0].set_xlabel('Gas')
    axs[1, 0].set_ylabel('Frequency')
    
    # 4. Transactions by Hour
    if 'datetime' in tx_df.columns:
        tx_df['hour'] = tx_df['datetime'].dt.hour
        hourly_tx = tx_df.groupby('hour').size()
        axs[1, 1].bar(hourly_tx.index, hourly_tx.values, color='blue', alpha=0.7)
        axs[1, 1].set_title('Transactions by Hour')
        axs[1, 1].set_xlabel('Hour of Day')
        axs[1, 1].set_ylabel('Number of Transactions')
    else:
        axs[1, 1].text(0.5, 0.5, 'Datetime information not available', 
                        horizontalalignment='center', verticalalignment='center')
    
    plt.tight_layout()
    plt.show()

def visualize_transaction_details(tx_df):
    """Additional visualizations to explore transaction details."""
    if tx_df.empty:
        print("No transaction data available for detailed visualization")
        return

    # Ensure addresses are treated as strings
    tx_df['from_address'] = tx_df['from_address'].astype(str)
    tx_df['to_address'] = tx_df['to_address'].astype(str)
    
    # 1. Top 10 Sender Addresses by Transaction Count
    top_senders = tx_df['from_address'].value_counts().head(10)
    
    # 2. Top 10 Receiver Addresses by Transaction Count
    top_receivers = tx_df['to_address'].value_counts().head(10)
    
    # 3. Scatter Plot: Transaction Value vs Gas Price
    plt.figure(figsize=(10, 6))
    plt.scatter(tx_df['value'], tx_df['gas_price'] / 1e9, alpha=0.5, color='teal')
    plt.xlabel('Transaction Value (ETH)')
    plt.ylabel('Gas Price (Gwei)')
    plt.title('Transaction Value vs Gas Price')
    plt.grid(True)
    plt.show()
    
    # 4. Bar Charts for Top Senders and Receivers
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    axs[0].bar(top_senders.index, top_senders.values, color='orange', alpha=0.7)
    axs[0].set_title('Top 10 Sender Addresses')
    axs[0].set_xlabel('Address')
    axs[0].set_ylabel('Number of Transactions')
    axs[0].tick_params(axis='x', rotation=45)
    
    axs[1].bar(top_receivers.index, top_receivers.values, color='purple', alpha=0.7)
    axs[1].set_title('Top 10 Receiver Addresses')
    axs[1].set_xlabel('Address')
    axs[1].set_ylabel('Number of Transactions')
    axs[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # 5. Cumulative Ether Transferred per Block
    cumulative = tx_df.groupby('block_number')['value'].sum().reset_index()
    plt.figure(figsize=(10,6))
    plt.plot(cumulative['block_number'], cumulative['value'], marker='o', linestyle='-')
    plt.xlabel('Block Number')
    plt.ylabel('Total Ether Transferred (ETH)')
    plt.title('Cumulative Ether Transferred per Block')
    plt.grid(True)
    plt.show()

def visualize_extra_data(tx_df):
    """Visualize extra data: most expensive transaction and smart contract creations."""
    if tx_df.empty:
        print("No transaction data available for extra analysis")
        return

    # 1. Most expensive transaction (highest ETH value)
    most_expensive_tx = tx_df.loc[tx_df['value'].idxmax()]
    print("Most Expensive Transaction:")
    display(most_expensive_tx)
    
    # 2. Smart contract creation transactions:
    # We assume these are transactions where 'to_address' is 'Contract Creation'
    contract_creations = tx_df[tx_df['to_address'] == 'Contract Creation']
    if not contract_creations.empty:
        print("\nSmart Contract Creation Transactions:")
        display(contract_creations)
        
        # Optional: Visualize number of smart contract creations per block
        sc_counts = contract_creations['block_number'].value_counts().sort_index()
        plt.figure(figsize=(10,6))
        plt.bar(sc_counts.index, sc_counts.values, color='magenta', alpha=0.7)
        plt.xlabel("Block Number")
        plt.ylabel("Number of Contract Creations")
        plt.title("Smart Contract Creations per Block")
        plt.grid(True)
        plt.show()
    else:
        print("No smart contract creation transactions found.")

def visualize_biggest_sender_receiver(tx_df):
    """Visualize the biggest sender and receiver by total ETH transferred."""
    if tx_df.empty:
        print("No transaction data available.")
        return

    # Calculate total ETH sent by each sender
    sender_totals = tx_df.groupby("from_address")["value"].sum().sort_values(ascending=False)
    biggest_sender = sender_totals.idxmax()
    biggest_sender_value = sender_totals.max()
    
    # Calculate total ETH received by each receiver (exclude contract creation transactions)
    receiver_totals = tx_df[tx_df["to_address"] != "Contract Creation"].groupby("to_address")["value"].sum().sort_values(ascending=False)
    biggest_receiver = receiver_totals.idxmax()
    biggest_receiver_value = receiver_totals.max()
    
    print("Biggest Sender (by total ETH sent):")
    print(f"Address: {biggest_sender}, Total ETH Sent: {biggest_sender_value:.4f}")
    print("\nBiggest Receiver (by total ETH received):")
    print(f"Address: {biggest_receiver}, Total ETH Received: {biggest_receiver_value:.4f}")
    
    # Plot top 5 senders and receivers by total ETH transferred
    top_senders = sender_totals.head(5)
    top_receivers = receiver_totals.head(5)
    
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    axs[0].bar(top_senders.index, top_senders.values, color='cyan', alpha=0.7)
    axs[0].set_title('Top 5 Senders by Total ETH Sent')
    axs[0].set_xlabel('Sender Address')
    axs[0].set_ylabel('Total ETH Sent')
    axs[0].tick_params(axis='x', rotation=45)
    
    axs[1].bar(top_receivers.index, top_receivers.values, color='magenta', alpha=0.7)
    axs[1].set_title('Top 5 Receivers by Total ETH Received')
    axs[1].set_xlabel('Receiver Address')
    axs[1].set_ylabel('Total ETH Received')
    axs[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

def dashboard():
    """Display blockchain data statistics and visualizations."""
    blocks_df, tx_df = load_blockchain_data()
    
    if blocks_df.empty:
        print("No block data available. Please crawl some blocks first.")
        return
    
    print("Blockchain Crawler Dashboard")
    print("-" * 30)
    print(f"Total blocks crawled: {len(blocks_df)}")
    print(f"Block range: {blocks_df['block_number'].min()} to {blocks_df['block_number'].max()}")
    print(f"Total transactions: {blocks_df['total_transactions'].sum()}")
    print(f"Average transactions per block: {blocks_df['total_transactions'].mean():.2f}")
    avg_util = (blocks_df['gas_used'] / blocks_df['gas_limit']).mean() * 100 if len(blocks_df) > 0 else 0
    print(f"Average gas utilization: {avg_util:.2f}%")
    print("-" * 30)
    
    print("Generating visualizations...")
    visualize_block_data(blocks_df)
    visualize_transaction_data(tx_df)
    print("Generating detailed transaction visualizations...")
    visualize_transaction_details(tx_df)
    print("Generating extra data visualizations...")
    visualize_extra_data(tx_df)
    print("Calculating biggest sender and receiver by total ETH transferred...")
    visualize_biggest_sender_receiver(tx_df)


In [4]:
# ==================== INTERACTIVE INTERFACE ====================
def create_interactive_dashboard():
    """Create an interactive dashboard using ipywidgets."""
    # Create widgets
    crawl_button = widgets.Button(description="Crawl Latest Blocks")
    num_blocks_input = widgets.IntSlider(value=5, min=1, max=40, description="# of Blocks:")
    
    start_listener_button = widgets.Button(description="Start Listener")
    stop_listener_button = widgets.Button(description="Stop Listener")
    stop_listener_button.disabled = True
    
    interval_input = widgets.IntSlider(value=10, min=5, max=60, description="Interval (s):")
    
    dashboard_button = widgets.Button(description="Show Dashboard")
    export_button = widgets.Button(description="Export Data")
    
    status_output = widgets.Output(layout={'border': '1px solid #ddd', 'min_height': '200px', 'max_height': '400px', 'overflow': 'auto'})
    
    # Layout for controls
    controls = widgets.VBox([
        widgets.HBox([num_blocks_input, crawl_button]),
        widgets.HBox([interval_input, start_listener_button, stop_listener_button]),
        widgets.HBox([dashboard_button, export_button]),
        status_output
    ])
    
    # Button event handlers
    def on_crawl_button_clicked(b):
        status_output.clear_output()
        with status_output:
            if web3 and web3.is_connected():
                print(f"Crawling {num_blocks_input.value} latest blocks...")
                crawl_latest_blocks(web3, num_blocks_input.value)
            else:
                print("Web3 connection not available. Please check your Infura API key.")
    
    crawler_running = [False]  # mutable flag
    listener_thread = [None]   # to store thread object
    
    def on_start_listener_clicked(b):
        if not web3 or not web3.is_connected():
            with status_output:
                print("Web3 connection not available. Please check your Infura API key.")
            return
        
        crawler_running[0] = True
        start_listener_button.disabled = True
        stop_listener_button.disabled = False
        
        status_output.clear_output()
        with status_output:
            print(f"Starting blockchain listener with {interval_input.value} second interval...")
            print("This will run until you click 'Stop Listener' or interrupt the kernel.")
            
            def run_listener():
                last_processed_block = web3.eth.block_number
                print(f"Starting from block {last_processed_block}")
                while crawler_running[0]:
                    try:
                        current_block = web3.eth.block_number
                        if current_block > last_processed_block:
                            print(f"New blocks detected! Processing blocks {last_processed_block+1} to {current_block}")
                            for block_num in range(last_processed_block + 1, current_block + 1):
                                print(f"Processing block {block_num}...")
                                block_data, transactions = fetch_block_data(web3, block_num)
                                if block_data and transactions:
                                    success = store_block_data(block_data, transactions)
                                    if success:
                                        print(f"Block {block_num} with {len(transactions)} transactions stored successfully")
                                    else:
                                        print(f"Failed to store data for block {block_num}")
                            last_processed_block = current_block
                        else:
                            print(f"{datetime.now().strftime('%H:%M:%S')} - No new blocks. Current block is still {current_block}.")
                        time.sleep(interval_input.value)
                    except Exception as e:
                        print(f"Error in listener: {e}")
                        time.sleep(interval_input.value)
                print("Listener stopped")
            
            # Start the listener in a background thread
            listener_thread[0] = threading.Thread(target=run_listener)
            listener_thread[0].daemon = True
            listener_thread[0].start()
    
    def on_stop_listener_clicked(b):
        crawler_running[0] = False
        start_listener_button.disabled = False
        stop_listener_button.disabled = True
        with status_output:
            print("Stopping blockchain listener...")
    
    def on_dashboard_button_clicked(b):
        status_output.clear_output()
        with status_output:
            dashboard()
    
    def on_export_button_clicked(b):
        try:
            blocks_df, tx_df = load_blockchain_data()
            export_time = datetime.now().strftime('%Y%m%d_%H%M%S')
            blocks_csv = f"blocks_export_{export_time}.csv"
            tx_csv = f"transactions_export_{export_time}.csv"
            blocks_df.to_csv(blocks_csv, index=False)
            tx_df.to_csv(tx_csv, index=False)
            with status_output:
                print("Data exported successfully!")
                print(f"Blocks data: {blocks_csv}")
                print(f"Transactions data: {tx_csv}")
        except Exception as e:
            with status_output:
                print(f"Error exporting data: {e}")
    
    # Wire up button events
    crawl_button.on_click(on_crawl_button_clicked)
    start_listener_button.on_click(on_start_listener_clicked)
    stop_listener_button.on_click(on_stop_listener_clicked)
    dashboard_button.on_click(on_dashboard_button_clicked)
    export_button.on_click(on_export_button_clicked)
    
    display(controls)


19:39:46 - No new blocks. Current block is still 22269371.


In [5]:
# ==================== MAIN PROGRAM ====================
def main():
    """Main function to run the blockchain crawler and interactive dashboard."""
    print("Lightweight Blockchain Crawler & Listener")
    print("=" * 40)
    
    # Setup database
    setup_database()
    
    # Connect to Ethereum and set the global web3 object
    global web3
    web3 = connect_to_ethereum()
    
    if not web3 or not web3.is_connected():
        print("Failed to connect to Ethereum. Please check your Infura API key or network settings.")
        return
    
    # Create the interactive dashboard
    create_interactive_dashboard()

# Run main() when this cell is executed.
main()


Lightweight Blockchain Crawler & Listener
Database setup complete
Successfully connected to Ethereum Sepolia Testnet
Current block number: 22269371
